<a href="https://colab.research.google.com/github/03Fede/Modelo-Predictivo/blob/main/C%C3%B3digo_Modelo_Predictivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Crear un conjunto de datos ficticio
np.random.seed(42)  # Asegurar reproducibilidad
data_ficticio = pd.DataFrame({
    'Edad': np.random.randint(18, 70, 200),
    'Nacionalidad': np.random.choice(['Argentina', 'Chile', 'México', 'EE.UU.', 'Brasil'], 200),
    'Tipo_Paquete': np.random.choice(['Económico', 'Premium', 'De lujo'], 200),
    'Dias_Viaje': np.random.randint(3, 21, 200),
    'Destino': np.random.choice(['Cancún', 'Bariloche', 'Río de Janeiro', 'Machu Picchu', 'Patagonia'], 200),
    'Calificacion': np.random.randint(1, 6, 200),
    'Cliente_Recurrente': np.random.choice([0, 1], 200, p=[0.7, 0.3])  # 30% de clientes recurrentes
})

# Separar características (X) y objetivo (y)
X = data_ficticio.drop(columns=['Cliente_Recurrente'])
y = data_ficticio['Cliente_Recurrente']

# Codificar variables categóricas
categorical_features = ['Nacionalidad', 'Tipo_Paquete', 'Destino']
# The 'sparse' argument is no longer supported, remove it.
encoder = OneHotEncoder(drop='first', handle_unknown='ignore')  # handle_unknown='ignore' for unseen values during prediction
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_features]).toarray(),  # Convert to dense array
                         columns=encoder.get_feature_names_out(categorical_features))

# Escalar variables numéricas
numerical_features = ['Edad', 'Dias_Viaje', 'Calificacion']
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X[numerical_features]), columns=numerical_features)

# Combinar datos procesados
X_prepared = pd.concat([X_scaled, X_encoded], axis=1)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y, test_size=0.3, random_state=42)

# Instanciar los modelos
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Entrenar y evaluar los modelos
results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)  # Entrenar
    y_pred = model.predict(X_test)  # Predecir en el conjunto de prueba
    accuracy = accuracy_score(y_test, y_pred)  # Calcular precisión
    results[model_name] = accuracy

# Mostrar resultados
for model_name, accuracy in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model_name}: Accuracy = {accuracy:.2f}")


Logistic Regression: Accuracy = 0.67
XGBoost: Accuracy = 0.62
Random Forest: Accuracy = 0.60
Decision Tree: Accuracy = 0.55


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:20:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
